In [1]:
import torch
from torch.autograd import Variable
from torch import nn, optim
from torch.nn import init

---

## 1. 标准RNN

In [2]:
# 建立简单的循环神经网络

basic_rnn = nn.RNN(input_size=3, hidden_size=5, num_layers=2)

In [3]:
# 访问第一层网络的 $w_{ih}$

basic_rnn.weight_ih_l0

Parameter containing:
 0.4127  0.1455 -0.2056
 0.2902 -0.0846 -0.0528
-0.2841 -0.4086 -0.2319
-0.3840 -0.1875 -0.1958
 0.3238  0.1001 -0.3414
[torch.FloatTensor of size 5x3]

In [4]:
# 访问第二层的 $w_{ih}$

basic_rnn.weight_ih_l1

Parameter containing:
 0.2278 -0.0144 -0.1155 -0.3075  0.2782
-0.3334 -0.3095 -0.4432 -0.0018 -0.0613
 0.4211  0.1219  0.1660  0.0539 -0.1025
-0.4322 -0.0808  0.4146  0.4229 -0.1143
 0.1132 -0.3710 -0.2455 -0.4088 -0.2012
[torch.FloatTensor of size 5x5]

In [5]:
# 访问第一层的 $w_{hh}$

basic_rnn.weight_hh_l1

Parameter containing:
 0.2312  0.0610 -0.1783 -0.0550 -0.4143
 0.0282 -0.0938  0.2335 -0.1646 -0.1390
-0.3874  0.1510  0.4280  0.0353 -0.4168
-0.3454  0.1570  0.2117  0.3527  0.0397
 0.1286  0.2015  0.2005  0.2474 -0.2706
[torch.FloatTensor of size 5x5]

In [6]:
# 访问第一层的 $b_{ih}$

basic_rnn.bias_ih_l0

Parameter containing:
 0.0946
 0.2652
 0.3729
-0.4259
-0.1166
[torch.FloatTensor of size 5]

In [7]:
# 输入的序列长度为5，批量是10，维度是3
toy_input = Variable(torch.randn(5, 10, 3))

# 初始隐藏状态 layer * direction = 2, 批量是10，维度是5
h_0 = Variable(torch.randn(2, 10, 5))

In [8]:
toy_output, h_n = basic_rnn(toy_input, h_0)

# toy_output的形状应该为 (5, 10, 5)
# h_n 的形状应该为 (2, 10, 5)
print(toy_output.shape)
print(h_n.shape)

torch.Size([5, 10, 5])
torch.Size([2, 10, 5])


---

## 2. LSTM

In [9]:
lstm = nn.LSTM(input_size=3, hidden_size=5, num_layers=2)

LSTM中间比标准RNN多了三个线性变换，多的三个线性变换的权重拼在一起，所以一共是4倍，同理偏置也是4倍。
换句话说，LSTM里面做了4个类似标准RNN所做的运算，所以参数个数是标准RNN的4倍。

In [10]:
# 参数的大小将变成 (4 * hidden_size, input_size) = (4 × 5, 3) = (20, 3)

lstm.weight_ih_l0

Parameter containing:
-0.1680  0.4076 -0.2904
 0.0665  0.2879 -0.4274
-0.1289 -0.0301  0.1010
 0.1326 -0.2075 -0.2184
 0.2738 -0.0784 -0.0595
-0.4395  0.3392  0.2053
 0.0411 -0.2497 -0.2467
 0.2173  0.4204  0.3379
-0.3309 -0.1578 -0.2533
-0.4292  0.0783 -0.1523
 0.4171 -0.1103  0.2153
 0.1660 -0.4352 -0.0984
-0.3791 -0.2864  0.1481
-0.2426 -0.3578  0.2702
 0.1799  0.0426  0.1215
-0.0611 -0.1987 -0.1964
-0.1658  0.3826 -0.3762
-0.1591 -0.0396 -0.1145
 0.0691 -0.1115  0.2626
-0.2222 -0.3675  0.0092
[torch.FloatTensor of size 20x3]

In [11]:
# toy_input的size为 (5, 10, 3)

lstm_out, (h_n, c_n) = lstm(toy_input)

In [12]:
# lstm的输出矩阵应该是 (5, 10, 5)
print('lstm_out_size = ', lstm_out.size())

# h_n 和 c_n 的size应为 (2, 10, 5)
print('h_n_size = ', h_n.size())
print('c_n_size = ', c_n.size())

lstm_out_size =  torch.Size([5, 10, 5])
h_n_size =  torch.Size([2, 10, 5])
c_n_size =  torch.Size([2, 10, 5])


---

## 3. GRU
1. GRU的隐藏状态数量为标准RNN的3倍；
2. 网络的隐藏状态不是 $h_0 \text{和} c_0$，而是只有 $h_0$；
3. 其余部分和LSTM相同；

---
## 4. 单步版本
Pytorch提供 `RNNCell`, `LSTMCell`, `GRUCell`分别作为这三个模型的单步版本。

它们的输入不再是一个序列，而是一个序列中的一步，也就是循环神经网络的一个循环。

单步版本在序列的应用上更加灵活，能在基础上添加更多的自定义操作。